<a href="https://colab.research.google.com/github/gergogomori/brain_wide_pes/blob/main/pes_spikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ONE-api
!pip install ibllib
!pip install iblatlas
!pip install numpy==1.26.4


In [ ]:
# RESTART SESSION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import shutil

from one.api import ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')

from iblatlas.atlas import AllenAtlas
brain_atlas = AllenAtlas()

from brainbox.io.one import SpikeSortingLoader

import warnings
import one.alf.exceptions as alferr
warnings.filterwarnings("ignore", category=alferr.ALFWarning)

# Load EID-s
eids = one.search(datasets='spikes.times.npy')
print('Number of sessions available:', len(eids))


In [ ]:
print(np.__version__)


# Load the current dicts

In [ ]:
with open('*** Update the folder path here to match your directory structure ***', 'rb') as f:
    trials_all = pickle.load(f)

with open('*** Update the folder path here to match your directory structure ***', 'rb') as f:
    probes_all = pickle.load(f)


# Parameter(s)

Keep these parameters!

In [ ]:
# Time period around movement onset
t_lims = (-500, 500) # ms


# Extracting the spikes aligned to the Go Cue

In [ ]:
spikes_all = {}

curr_eids = list(set(trials_all.keys()) & set(probes_all.keys()))

for ind_EID, EID in enumerate(curr_eids):

    print(f'Done with {np.round(ind_EID / len(curr_eids) * 100, 2)} % of all sessions.')

    # Load trial data
    trials = one.load_object(EID, 'trials')

    curr_pids = sorted(list(probes_all[EID].keys()))

    for contr in sorted(list(trials_all[EID].keys())):

        spikes_all[(EID, contr)] = {}

        curr_trials = sorted(list(trials_all[EID][contr].keys()))

        for PID in curr_pids:

            # Load spiketrains
            sl = SpikeSortingLoader(pid=PID, one=one, atlas=brain_atlas)
            spikes, clusters, channels = sl.load_spike_sorting()
            clusters = sl.merge_clusters(spikes, clusters, channels)

            # Selecting only the spikes which have a proper value
            valid_i        = np.logical_and(np.isfinite(spikes['times']), np.isfinite(spikes['depths']))
            valid_spikes   = spikes['times'][valid_i]
            valid_clusters = spikes['clusters'][valid_i]

            # Selecting the "good" clusters
            target_clusts = sorted(list((probes_all[EID][PID])))
            i_tar_clust   = np.isin(valid_clusters, target_clusts)
            tar_spikes    = valid_spikes[i_tar_clust]
            tar_clusters  = valid_clusters[i_tar_clust]

            for trial in curr_trials:

                t_align = trials['goCue_times'][trial]
                t_pre   = t_lims[0]
                t_post  = t_lims[1]

                t_min = t_align + (t_pre  / 1000)
                t_max = t_align + (t_post / 1000)

                # Filter out spikes outside of the interesting time period of the current trial
                curr_inds     = np.logical_and(tar_spikes > t_min, tar_spikes < t_max)
                curr_spikes   = tar_spikes[curr_inds]
                curr_clusters = tar_clusters[curr_inds]

                for neuron in target_clusts:

                    if (PID, neuron) not in spikes_all[(EID, contr)].keys():
                        spikes_all[(EID, contr)][(PID, neuron)] = {}

                    # Filter out spikes from other neurons
                    spikes_all[(EID, contr)][(PID, neuron)][trial] = np.subtract(curr_spikes[np.equal(curr_clusters, neuron)], t_align) * 1000

    shutil.rmtree('/root/Downloads/ONE/openalyx.internationalbrainlab.org/' + one.get_details(EID)['lab'], ignore_errors=True)

with open('*** Update the folder path here to match your directory structure ***', 'wb') as f:
    pickle.dump(spikes_all, f, pickle.HIGHEST_PROTOCOL)
